# Word2Vec for Text Classification

In this short notebook, we will see an example of how to use a pre-trained Word2vec model for doing feature extraction and performing text classification.

We will use the sentiment labelled sentences dataset from UCI repository
http://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences

The dataset consists of 1500 positive, and 1500 negative sentiment sentences from Amazon, Yelp, IMDB. Let us first combine all the three separate data files into one using the following unix command:

```cat amazon_cells_labelled.txt imdb_labelled.txt yelp_labelled.txt > sentiment_sentences.txt```

For a pre-trained embedding model, we will use the Google News vectors.
https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM

Let us get started!

In [1]:
# basic imports
import os
from time import time

# pre-processing imports
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

# imports related to modeling
import numpy as np
from gensim.models import Word2Vec, KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

#### 1. load word embeddings

In [2]:
#Load the pre-trained word2vec model and the dataset
data_path = "/Users/chenwang/Workspace/github/practical_nlp/chapter4_text_classification/data/sentiment_labelled_sentences"
model_path = "/Users/chenwang/Workspace/datasets/models"
path_to_model = os.path.join(model_path,'GoogleNews-vectors-negative300.bin')
training_data_path = os.path.join(data_path, "sentiment_sentences.txt")



In [3]:

#Load W2V model. This will take some time. 
%time w2v_model = KeyedVectors.load_word2vec_format(path_to_model, binary=True)
print('done loading Word2Vec')



CPU times: user 1min 34s, sys: 6.36 s, total: 1min 41s
Wall time: 1min 49s
done loading Word2Vec


#### 2. 读入原始数据，构建文本集和标签

In [4]:
# Read text data, cats.
# the file path consists of tab separated sentences and cats.
texts = []
cats = []
fh = open(training_data_path)
for line in fh:
    text, sentiment = line.split("\t")
    texts.append(text)
    cats.append(sentiment)

In [5]:
# Inspect the model
word2vec_vocab = w2v_model.vocab.keys()
word2vec_vocab_lower = [item.lower() for item in word2vec_vocab]
print(len(word2vec_vocab))

3000000


In [6]:
# Inspect the dataset
print(len(cats), len(texts))
print(texts[1])
print(cats[1])

3000 3000
Good case, Excellent value.
1



#### 3. 预处理：移除stop words + digits

In [7]:
# preprocess the text.
# remove stop words and digits

def preprocess_corpus(texts):
    mystopwords = set(stopwords.words("english"))
    def remove_stops_digits(tokens):
        # Nested function that lowercases, removes stopwords and digits from a list of tokens
        return [token.lower() for token in tokens if token not in mystopwords and not token.isdigit()
               and token not in punctuation]
    #This return statement below uses the above function to process twitter tokenizer output further. 
    return [remove_stops_digits(word_tokenize(text)) for text in texts]



In [8]:
texts_processed = preprocess_corpus(texts)
print(len(cats), len(texts_processed))
print(texts_processed[1])
print(cats[1])

3000 3000
['good', 'case', 'excellent', 'value']
1



#### 4. 计算每个document 的embedding -- document 中所有word embedding 的平均值

In [18]:
# Creating a feature vector by averaging all embeddings for all sentences

def embedding_feats(list_of_lists):
    DIMENSION = 300
    zero_vector = np.zeros(DIMENSION)
    feats = []
    for tokens in list_of_lists:
        feat_for_this =  np.zeros(DIMENSION)
        count_for_this = 0
        for token in tokens:
            if token in w2v_model:
                feat_for_this += w2v_model[token]
                count_for_this +=1
        if count_for_this == 0:  # 如果没有valid token，放一个全零的向量
            print('Error: no valid token in the document: ', tokens)
            feats.append(feat_for_this) 
        else:
            feats.append(feat_for_this/count_for_this)  # 一个document 的vector 是每个单词vector 相加的平均值        
    return feats


In [19]:
train_vectors = embedding_feats(texts_processed)
print(len(train_vectors))

Error: no valid token in the document:  ['10/10']
Error: no valid token in the document:  ['10/10']
3000


In [20]:
# Take any classifier (LogisticRegression here, and train/test it like before.
classifier = LogisticRegression(random_state=1234)

# Training and testing data set split
train_data, test_data, train_cats, test_cats = train_test_split(train_vectors, cats)


In [21]:
classifier.fit(train_data, train_cats)
print("Accuracy: ", classifier.score(test_data, test_cats))
preds = classifier.predict(test_data)
print(classification_report(test_cats, preds))

/Users/chenwang/anaconda3/envs/tf/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy:  0.848
              precision    recall  f1-score   support

          0
       0.86      0.84      0.85       386
          1
       0.83      0.86      0.85       364

   micro avg       0.85      0.85      0.85       750
   macro avg       0.85      0.85      0.85       750
weighted avg       0.85      0.85      0.85       750



Not bad. With little efforts we got 84% accuracy. Thats a great starting model to have!!